##Connect to cloud

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##Trees

In [2]:
from google.colab import files
files.upload() 

Saving tree.py to tree.py


{'tree.py': b'import copy\r\nimport pandas as pd\r\nimport networkx as nx\r\nimport numpy as np\r\nimport matplotlib.pyplot as plt\r\nimport pickle\r\nimport os\r\n\r\nGOAL = "demand"\r\n# GOAL = "Goal"\r\n\r\n\r\nclass Node:\r\n    def __init__(self, data, choices=None, depth=0):\r\n        self.data = data\r\n        self.choices = choices\r\n        self.children = []\r\n        self.depth = depth\r\n\r\n    def __del__(self):\r\n        for child in self.children:\r\n            del child\r\n        del self.children\r\n\r\n    def add_children(self, children):\r\n        self.children = children\r\n\r\n\r\nclass Tree:\r\n    def __init__(self, records_df, limit=0, attributes=None, goal=GOAL, name=""):\r\n        """creates the tree based on a dictionary of attributes and options"""\r\n        self.limit = limit\r\n        self.goal = goal\r\n        self.name = name\r\n        if records_df is not None:\r\n            self.rows = len(records_df)\r\n            self.root = self.cre

##Constants

In [0]:
import pandas as pd
import math
from multiprocessing import Pool
from tree import *

In [0]:
PATH_TO_FOLDER = "/content/gdrive/My Drive/AI_project/"
INPUT_PATH = "Data/training_data_1.csv"
TEST_PATH = "Data/test_data_1.csv"
TRAINING_SET = "1"

GOAL = "demand"
TRAIN_LEVEL = 0.5
POOL_SIZE = 4
TREE = "Tree"
ENTROPY = "EntropyTree"
INFORMATION_GAIN = "InformationGainTree"
INFORMATION_RATIO = "InformationRatioTree"

BASIC_ATTRIBUTES = ["L1", "L2", "time"]
IGNORE_LIST = BASIC_ATTRIBUTES + ["Unnamed: 0", "Unnamed: 0.1", "cluster_id",
                                  "thunderstorm", "foggy", "humidity", "demand"]

##Tree Creation

In [0]:
def tree_creation(type, records_df, limit=0, attributes=None, goal=GOAL, name=""):
    """This function creates the trees"""
    if type == TREE:
        return Tree(records_df, limit, attributes, goal, name)
    elif type == ENTROPY:
        return EntropyTree(records_df, limit, attributes, goal, name)
    elif type == INFORMATION_GAIN:
        return InformationGainTree(records_df, limit, attributes, goal, name)
    elif type == INFORMATION_RATIO:
        return InformationRatioTree(records_df, limit, attributes, goal, name)
    else:
        return


def create_attributes_list(data):
    """Creates a list of attributes in order to build th trees"""
    attributes_list = []
    for column in data.columns:
        if column not in IGNORE_LIST:
            attributes_list.append(BASIC_ATTRIBUTES + [column])
    return attributes_list


def get_type(tree):
    """Gets the type of the tree"""
    return str(type(tree)).split('.')[1].split('\'')[0]


def create_trees(training_data, goal):
    p = Pool(POOL_SIZE)
    all_trees = []
    # every tree we want to create has to come in the format of
    # (type, df, limit, attributes, goal)
    attr_list = create_attributes_list(training_data)
    for lst in attr_list:
        print(lst[-1])
        trees = [(TREE, training_data, 0, lst, goal, TREE + "_" + lst[-1] +
                  "_" + TRAINING_SET)]
        trees.append((ENTROPY, training_data, 0, lst, goal,
                      ENTROPY + "_" + lst[-1] + "_" + TRAINING_SET))
        trees.append((INFORMATION_GAIN, training_data, 0, lst, goal,
                      INFORMATION_GAIN + "_" + lst[-1] + "_" + TRAINING_SET))
        trees.append((INFORMATION_RATIO, training_data, 0, lst, goal,
                      INFORMATION_RATIO + "_" + lst[-1] + "_" + TRAINING_SET))
        res = p.starmap(tree_creation, trees)
        for t in res:
            t.save_tree(PATH_TO_FOLDER + "Trees/" + t.name + ".txt")
            all_trees.append(t)
    p.close()
    p.join()
    return all_trees


def export_training_and_test(training_data, test_data):
    """This functino saves the training data and the testing data"""
    training_data.to_csv("training_data.csv")
    test_data.to_csv("test_data.csv")
    
def load_data(path):
    """"""
    return pd.read_csv(path)

In [0]:
def additional_trees(training_data, goal):
    """"""
    p = Pool(POOL_SIZE)
    # every tree we want to create has to come in the format of
    # (type, df, limit, attributes, goal)
    all_trees = []
    new_basic = BASIC_ATTRIBUTES + ["weekday"]
    attr_list = [new_basic + ["month"]]
    attr_list.append(new_basic + ["clear_sky"])
    attr_list.append(new_basic + ["extreme_weather"])
#     attr_list.append(new_basic + ["clear_sky", "extreme_weather"])
    for lst in attr_list:
        filename = "_".join(list(set(lst) - set(BASIC_ATTRIBUTES)))
        print(filename)
        trees = [(TREE, training_data, 0, lst, goal,
                  TREE + "_" + filename + "_" + TRAINING_SET)]
        trees.append((ENTROPY, training_data, 0, lst, goal,
                      ENTROPY + "_" + filename + "_" + TRAINING_SET))
        trees.append((INFORMATION_GAIN, training_data, 0, lst, goal,
                      INFORMATION_GAIN + "_" + filename + "_" + TRAINING_SET))
        trees.append((INFORMATION_RATIO, training_data, 0, lst, goal,
                      INFORMATION_RATIO + "_" + filename + "_" + TRAINING_SET))
        res = p.starmap(tree_creation, trees)
        for t in res:
            t.save_tree(PATH_TO_FOLDER + "Trees/" + t.name + ".txt")
            all_trees.append(t)
    p.close()
    p.join()
    return all_trees
    

In [0]:
    training_data = pd.read_csv(PATH_TO_FOLDER + INPUT_PATH)
    # create trees based on training data
    all_trees = create_trees(training_data, GOAL)
    print("finish normal trees")
    all_trees.extend(additional_trees(training_data, GOAL))

##Load all trees

In [0]:
from os import listdir
from os.path import isfile, join

def get_tree_files():
    onlyfiles = [f for f in listdir(PATH_TO_FOLDER + "Trees") if
             isfile(join(PATH_TO_FOLDER + "Trees", f)) and TRAINING_SET in f]
    return onlyfiles


def load_all_trees():
    files = get_tree_files()
    path = PATH_TO_FOLDER + "Trees/"
#     name = "_clear_sky_weekday_extreme_weather_1.txt"
#     files = [path + TREE + name, path + ENTROPY + name, path + INFORMATION_GAIN + name, path + INFORMATION_RATIO + name]
    trees = []
    for file in files:
        new_tree = Tree(None)
        new_tree.load_tree(join(PATH_TO_FOLDER + "Trees", file))
        trees.append(new_tree)
    return trees

In [0]:
all_trees = load_all_trees()
len(all_trees)

##Create file for NN

In [0]:
TEST_START = "Subsets/test_data_"

def create_file(test_data, all_trees, goal, part):
    """This function generates a file with the results of each tree and the
    actual result per line in the test data"""
    columns = [t.name for t in all_trees]
    columns.append(goal)
    output = pd.DataFrame(columns=columns)
#     for i in range(len(test_data)):
#         print(str(i) + " out of " + str(len(test_data)), len(output))
    while not test_data.empty:
        test_data, row = test_data.iloc[1:], test_data.head(1)
        print(part, len(test_data), len(output))
        row_dict = {t.name: t.get_val(row) for t in all_trees}
        row_dict[goal] = row[goal][0]
        output = output.append(pd.DataFrame.from_dict([row_dict]))
        del row
    if not output.empty:
        output.to_csv(PATH_TO_FOLDER + "Data/testing_by_tree_" + TRAINING_SET + "_" + str(part) + ".csv")
        
def change_numeric_to_format(data):
    for col in data.columns:
        if np.issubdtype(data[col].dtype, np.floating):
            data[col] = data[col].apply(lambda x: int(x))
        else:
            continue


def generate_file_for_part(part):
    test_data = pd.read_csv(PATH_TO_FOLDER + TEST_START + TRAINING_SET + "_" + part + ".csv")
    change_numeric_to_format(test_data)
    create_file(test_data, all_trees, GOAL, part)
    del test_data

In [0]:
start = 0
end = 19
test_range = []
for i in range(start,end+1):
    test_range.append("0" * (4 - len(str(i))) + str(i))

In [11]:
# TEST_RANGE = ["0001", "0002", "0003", "0004"]

p = Pool(20)

p.map(generate_file_for_part, test_range)

p.close()
p.join()

0003 1230 0
0002 1229 0
0010 1230 0
0011 1230 0
0004 1229 0
0009 1229 0
0012 1229 0
0016 1230 0
0017 1229 0
0005 1230 0
0019 1229 0
0018 1230 0
0007 1229 0
0000 1229 0
0008 1230 0
0013 1230 0
0014 1229 0
0006 1230 0
0015 1230 0
0001 1230 0


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True